In [1]:
import json
import re

def extract_posts_from_damaged_json(filename):
    """손상된 JSON 파일에서 post 객체들을 추출"""
    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()
    
    posts = []
    
    # "sourceId"로 시작하는 모든 객체 찾기
    # 각 post는 "sourceId"로 시작
    pattern = r'"sourceId":\s*"([^"]+)"'
    matches = list(re.finditer(pattern, content))
    
    print(f"총 {len(matches)}개의 sourceId 발견")
    
    # 각 sourceId 위치에서 완전한 객체 추출 시도
    for i, match in enumerate(matches):
        start_pos = match.start()
        
        # 객체 시작 위치 찾기 (이전 { 찾기)
        obj_start = content.rfind('{', 0, start_pos)
        
        # 다음 객체 시작 또는 배열 끝까지
        if i < len(matches) - 1:
            next_start = matches[i + 1].start()
            obj_end_search = content.rfind('}', start_pos, next_start)
        else:
            obj_end_search = content.rfind('}', start_pos)
        
        if obj_start != -1 and obj_end_search != -1:
            obj_text = content[obj_start:obj_end_search + 1]
            
            try:
                # JSON 파싱 시도
                obj = json.loads(obj_text)
                posts.append(obj)
            except json.JSONDecodeError:
                # 개별 필드 추출 방식으로 fallback
                try:
                    post = extract_post_fields(obj_text)
                    if post:
                        posts.append(post)
                except:
                    print(f"  경고: {i+1}번째 객체 추출 실패")
                    continue
    
    return posts

def extract_post_fields(text):
    """정규식으로 각 필드 추출"""
    post = {}
    
    # 각 필드 추출
    patterns = {
        'sourceId': r'"sourceId":\s*"([^"]*)"',
        'targetBoardId': r'"targetBoardId":\s*"([^"]*)"',
        'createdAt': r'"createdAt":\s*"([^"]*)"',
        'updatedAt': r'"updatedAt":\s*"([^"]*)"',
        'inlineHandling': r'"inlineHandling":\s*"([^"]*)"',
        'title': r'"title":\s*"([^"]*)"',
        'readCount': r'"readCount":\s*"([^"]*)"',
        'status': r'"status":\s*"([^"]*)"',
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            post[key] = match.group(1)
    
    # writer 객체
    email_match = re.search(r'"emailId":\s*"([^"]*)"', text)
    if email_match:
        post['writer'] = {'emailId': email_match.group(1)}
    
    # content 추출 (복잡하므로 간단히 처리)
    content_match = re.search(r'"content":\s*"(.*?)"\s*,\s*"status"', text, re.DOTALL)
    if content_match:
        post['content'] = content_match.group(1)
    else:
        post['content'] = ""
    
    # attaches 배열
    post['attaches'] = []
    
    return post if 'sourceId' in post else None

# 메인 실행
print("JSON 파일 파싱 중...")
posts = extract_posts_from_damaged_json('board_targetBoardId_insert.json')

print(f"\n✓ 총 {len(posts)}개의 게시물 추출 완료")

# addPost 형식으로 변환하여 저장
output_path = 'converted_boards.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    for post in posts:
        post_json = json.dumps(post, ensure_ascii=False, separators=(',', ':'))
        f.write(f'addPost{post_json}\n')

print(f"✓ 변환 완료: {output_path}")
print(f"\n첫 번째 게시물 미리보기:")
if posts:
    first_post = json.dumps(posts[0], ensure_ascii=False, indent=2)
    print(first_post[:500] + "...")

JSON 파일 파싱 중...
총 1750개의 sourceId 발견

✓ 총 1750개의 게시물 추출 완료
✓ 변환 완료: converted_boards.txt

첫 번째 게시물 미리보기:
{
  "sourceId": "_4102705",
  "targetBoardId": "1427170574110560256",
  "createdAt": "1762762585000",
  "updatedAt": "1762762585000",
  "inlineHandling": "True",
  "writer": {
    "emailId": "accounting"
  },
  "title": "제1회 대한민국 인공지능 혁신대상- “종합대상” 수상",
  "readCount": "40",
  "content": "<div class=\"dispNone\" id=\"atcl_cont\" name=\"editorHideContent\"><p><span style=\"font-size: 12pt;\">어제 국회에서 개최된 <span style=\"font-weight: bold;\">“제1회 대한민국 인공지능 혁신대상”</span> 수상식에서 </span></p>\n<p><span style...
